In [1]:
# Install necessary packages

from Bio import SeqIO
from Bio.Data import CodonTable
import pandas as pd

In [6]:
# Dr. X's file calling function
# I changed the name of the function from "get_sequences_from_file' to "get_seq" because the original name was long

def get_seq(fasta_fn):                                         # def defines the following as a function. Function given name. fasta_fn is the single argument of the function
    sequence_data_dict = {}                                    # creates an empty dictionary
    for record in SeqIO.parse(fasta_fn, "fasta"):              # SeqIO reads SeqRecords. "fasta" tells it that the file in question is in FASTA format
                                                               # SeqIO.parse separates the multiple FASTA records in one file
                                                               # for loop loops through every record from the FASTA argument file parsed out and separated by SeqIO
        description = record.description.split()               # split the description metadata line that starts the FASTA record into a list of strings
        species_name = description[1] + " " + description[2]   # variable "species_name" is set to the 1st and 2nd items in "description" for the current record
        sequence_data_dict[species_name] = record.seq          # add the species name and the sequence of the current record to the dictionary
    return(sequence_data_dict)                                 # output the contents of the dictionary
 
get_seq("bears_cytb.fasta")                                    # call get_seq function with the bear cytochrome C fasta file as argument, importing sequence data

SyntaxError: invalid syntax (<ipython-input-6-a1d0be515d3e>, line 9)